# Poison Certified Training on UCI Datasets

In this notebook, we'll train a simple neural network with 1 hidden layer on the UCI-houseelectric regression task. We'll use Abstract Gradient Training to compute bounds on the parameters of the model under a potential data poisoning attack.

In [1]:
%load_ext autoreload
%autoreload 2
import torch
import abstract_gradient_training as agt
from abstract_gradient_training.bounded_models import IntervalBoundedModel

import uci_datasets  # python -m pip install git+https://github.com/treforevans/uci_datasets.git
torch.manual_seed(0)

### 1. Load the data

In [2]:
data = uci_datasets.Dataset("houseelectric")
x_train, y_train, x_test, y_test = data.get_split(split=0)

# Normalise the features and labels
x_train_mu, x_train_std = x_train.mean(axis=0), x_train.std(axis=0)
x_train = (x_train - x_train_mu) / x_train_std
x_test = (x_test - x_train_mu) / x_train_std
y_train_min, y_train_range = y_train.min(axis=0), y_train.max(axis=0) - y_train.min(axis=0)
y_train = (y_train - y_train_min) / y_train_range
y_test = (y_test - y_train_min) / y_train_range

# Form datasets and dataloaders
train_data = torch.utils.data.TensorDataset(torch.from_numpy(x_train).float(), torch.from_numpy(y_train).float())
test_data = torch.utils.data.TensorDataset(torch.from_numpy(x_test).float(), torch.from_numpy(y_test).float())
train_loader = torch.utils.data.DataLoader(train_data, batch_size=20000, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1000, shuffle=False)

houseelectric dataset, N=2049280, d=11


### 2. Configure the poisoning attack and training parameters

We'll bound the following feature-poisoning attack: in each batch, up to `k_poison=200` data points can be poisoned
by up to `epsilon=0.01` in the $\ell_\infty$-norm.


In [3]:
# configure the training parameters
config = agt.AGTConfig(
    fragsize=20000,
    learning_rate=0.005,
    epsilon=0.01,
    k_poison=200,
    n_epochs=1,
    device="cuda:1",
    loss="mse",
    log_level="DEBUG",
)

### 3. Initialize the model

In [4]:
# get the nn model
model = torch.nn.Sequential(torch.nn.Linear(11, 64), torch.nn.ReLU(), torch.nn.Linear(64, 1)).to(config.device)
# we'll wrap this model in a bounded version that can compute the bounds on the outputs and gradients required for AGT
bounded_model = IntervalBoundedModel(model)

### 4. Train the model using AGT

At each iteration, AGT will report the current MSE plus its certified bounds. Logging can be controlled via the `log_level` parameter in the config.

In [5]:
agt. poison_certified_training(bounded_model, config, train_loader, test_loader)

[AGT] [INFO    ] [12:22:27] =================== Starting Poison Certified Training ===================
[AGT] [DEBUG   ] [12:22:27] 	Optimizer params: n_epochs=1, learning_rate=0.005, l1_reg=0.0, l2_reg=0.0
[AGT] [DEBUG   ] [12:22:27] 	Learning rate schedule: lr_decay=0.0, lr_min=0.0
[AGT] [DEBUG   ] [12:22:27] 	Gradient clipping: gamma=inf, method=clamp
[AGT] [DEBUG   ] [12:22:27] 	Gradient noise: type=gaussian, multiplier=0
[AGT] [DEBUG   ] [12:22:27] 	Adversary feature-space budget: epsilon=0.01, k_poison=200
[AGT] [DEBUG   ] [12:22:27] 	Adversary label-space budget: label_epsilon=0, label_k_poison=0, poison_target_idx=-1
[AGT] [DEBUG   ] [12:22:27] 	Paired poisoning: False
[AGT] [INFO    ] [12:22:27] Starting epoch 1


[AGT] [DEBUG   ] [12:22:28] Initialising dataloader batchsize to 20000
[AGT] [INFO    ] [12:22:28] Batch 1. Loss (mse): 0.201 <= 0.201 <= 0.201
[AGT] [INFO    ] [12:22:28] Batch 2. Loss (mse): 0.182 <= 0.182 <= 0.182
[AGT] [INFO    ] [12:22:28] Batch 3. Loss (mse): 0.165 <= 0.166 <= 0.166
[AGT] [INFO    ] [12:22:29] Batch 4. Loss (mse): 0.153 <= 0.154 <= 0.154
[AGT] [INFO    ] [12:22:29] Batch 5. Loss (mse): 0.132 <= 0.132 <= 0.133
[AGT] [INFO    ] [12:22:29] Batch 6. Loss (mse): 0.130 <= 0.131 <= 0.131
[AGT] [INFO    ] [12:22:29] Batch 7. Loss (mse): 0.116 <= 0.117 <= 0.118
[AGT] [INFO    ] [12:22:29] Batch 8. Loss (mse): 0.107 <= 0.107 <= 0.108
[AGT] [INFO    ] [12:22:29] Batch 9. Loss (mse): 0.094 <= 0.095 <= 0.096
[AGT] [INFO    ] [12:22:30] Batch 10. Loss (mse): 0.092 <= 0.093 <= 0.094
[AGT] [INFO    ] [12:22:30] Batch 11. Loss (mse): 0.087 <= 0.088 <= 0.089
[AGT] [INFO    ] [12:22:30] Batch 12. Loss (mse): 0.082 <= 0.084 <= 0.085
[AGT] [INFO    ] [12:22:30] Batch 13. Loss (mse): 

IntervalBoundedModel(
	self.modules=[
		Linear(in_features=11, out_features=64, bias=True)
		ReLU()
		Linear(in_features=64, out_features=1, bias=True)
	],
	self.interval_matmul='rump',
	self.trainable=True,
)

### 5. Compute bounds on the final MSE of the model on the test set

In [6]:
# evaluate the trained model
test_batch, test_labels = next(iter(test_loader))
mse = agt.test_metrics.test_mse(bounded_model, test_batch, test_labels)
print(f"Test MSE: nominal = {mse[1]:.4g}, certified upper bound = {mse[0]:.4g}, certified lower bound = {mse[2]:.4g}")

Test MSE: nominal = 0.02631, certified upper bound = 1.473, certified lower bound = 0
